In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk
from sklearn.model_selection import train_test_split

# load the dataset
df = pd.read_csv('./data/train.csv')
print(df.head())
print(df.shape)


   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...
(27643, 3)


In [2]:
X_train = df.loc[:19999, :]
print(X_train.shape)
X_valid = df.loc[20000:, :]
print(X_valid.shape)
X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)

(20000, 3)
(7643, 3)


In [22]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    soup = soup.find('footer',{'class':'article-topics'})
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    porter = PorterStemmer()
    res = [porter.stem(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, df.shape[0]):
    res.append(get_category(df['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
res

[['u.s.', 'space', 'world', 'challeng', 'earth', 'asteroid'],
 ['googl',
  'tech',
  'open sourc',
  'u.s.',
  'apps and softwar',
  'patent lawsuit theat',
  'opn pledg',
  'software pat'],
 ['sport', 'entertain', 'nfl draft', 'nfl', 'televis'],
 ['video', 'watercool', 'sport'],
 ['instagram video', 'sport', 'instagram', 'entertain', 'nfl'],
 ['lobbyist',
  'govern',
  'u.s.',
  'startup',
  'lobbi',
  'internet',
  'mobil',
  'polit',
  'internet service provid',
  'world'],
 ['music', 'iggy azalea', 'entertain', 'parodi', 'video', 'funni'],
 ['bill cosbi', 'televis', 'entertain'],
 ['vending machin', 'tech', 'apps and softwar', 'mobil', 'gadget'],
 ['busi', 'media', 'paywal', 'the new york tim'],
 ['astronomi', 'moon', 'space', 'world', 'saturn'],
 ['mad men', 'humor', 'photographi', 'televis', 'watercool'],
 ['u.s.', 'apps and softwar', 'tech', 'nevada'],
 ['busi', 'u.s.', 'appl', 'dev & design', 'steve job'],
 ['soccer', 'sport', 'world cup', "women's world cup", 'uswnt', 'enterta

In [3]:
pop = X_train[X_train.loc[:,"Popularity"] > 0]
unpop = X_train[X_train.loc[:,"Popularity"] < 0]
pop = pop.reset_index(drop=True)
unpop = unpop.reset_index(drop=True)
pop

,Id,Popularity,Page content
0,1,1,"<html><head><div class=""article-info""><span cl..."
1,2,1,"<html><head><div class=""article-info""><span cl..."
2,6,1,"<html><head><div class=""article-info""><span cl..."
3,8,1,"<html><head><div class=""article-info""><span cl..."
4,11,1,"<html><head><div class=""article-info""><span cl..."
...,...,...,...
9823,19984,1,"<html><head><div class=""article-info""><span cl..."
9824,19988,1,"<html><head><div class=""article-info""><span cl..."
9825,19993,1,"<html><head><div class=""article-info""><span cl..."
9826,19995,1,"<html><head><div class=""article-info""><span cl..."


In [17]:
print(pop.shape)

(9828, 3)


In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer

stop_words = set(stopwords.words("english"))

def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    t = article.get_text().split(' ')
    porter = PorterStemmer()
    res = [porter.stem(word) for word in t if not word in  
        stop_words]
    return res

pop_cat = []
for i in range(0, pop.shape[0]):
    pop_cat.extend(get_article(pop['Page content'][i]))

In [6]:
pop_cat[:100]

['',
 'googl',
 'took',
 'stand',
 'sort',
 'patent-lawsuit',
 'theater',
 'thursday',
 'new',
 'open',
 'patent',
 'non-assert',
 '(opn)',
 'pledge.',
 '',
 'As',
 'explain',
 "google'",
 'duan',
 'valz,',
 'opn',
 'pledge,',
 'googl',
 'promis',
 '"not',
 'sue',
 'user,',
 'distributor',
 'develop',
 'open-sourc',
 'softwar',
 'specifi',
 'patents,',
 'unless',
 'first',
 'attacked."',
 'now,',
 'googl',
 'make',
 'patent',
 'avail',
 'others.',
 'instead,',
 'start',
 'small',
 '10',
 'patent',
 'focus',
 'mapreduce,',
 'program',
 'model',
 'handl',
 'larg',
 'data',
 'sets.',
 'there',
 'alreadi',
 'open-sourc',
 'version',
 'mapreduc',
 'avail',
 '—',
 'includ',
 'hadoop',
 '—',
 'wide',
 'use',
 'across',
 'internet.',
 'googl',
 'say',
 'time,',
 'plan',
 'extend',
 'opn',
 'pledg',
 'googl',
 'patents.',
 'see',
 'also:',
 '10',
 'interest',
 'googl',
 'glass',
 'winner',
 'their',
 'wacki',
 'idea',
 '',
 'the',
 'big',
 'caveat',
 'googl',
 'pledg',
 'sue',
 'anyon',
 'use',

In [7]:
unpop_cat = []
for i in range(0, unpop.shape[0]):
    unpop_cat.extend(get_article(unpop['Page content'][i]))

In [8]:
pop_dict = {}

for cat in pop_cat:
    if cat in pop_dict:
        pop_dict[cat] = pop_dict[cat] + 1
    else:
        pop_dict[cat] = 1
pop_dict

{'': 558252,
 'googl': 4708,
 'took': 1633,
 'stand': 1157,
 'sort': 591,
 'patent-lawsuit': 1,
 'theater': 293,
 'thursday': 657,
 'new': 15211,
 'open': 2548,
 'patent': 222,
 'non-assert': 1,
 '(opn)': 1,
 'pledge.': 6,
 'As': 2562,
 'explain': 930,
 "google'": 532,
 'duan': 12,
 'valz,': 1,
 'opn': 6,
 'pledge,': 5,
 'promis': 579,
 '"not': 105,
 'sue': 58,
 'user,': 87,
 'distributor': 22,
 'develop': 2998,
 'open-sourc': 75,
 'softwar': 1079,
 'specifi': 98,
 'patents,': 10,
 'unless': 256,
 'first': 6822,
 'attacked."': 2,
 'now,': 1199,
 'make': 10051,
 'avail': 2276,
 'others.': 229,
 'instead,': 401,
 'start': 3756,
 'small': 1691,
 '10': 2941,
 'focus': 692,
 'mapreduce,': 1,
 'program': 1448,
 'model': 1070,
 'handl': 441,
 'larg': 1174,
 'data': 2548,
 'sets.': 31,
 'there': 1515,
 'alreadi': 2079,
 'version': 1769,
 'mapreduc': 3,
 '—': 21373,
 'includ': 4913,
 'hadoop': 2,
 'wide': 597,
 'use': 11708,
 'across': 1639,
 'internet.': 214,
 'say': 5867,
 'time,': 1177,
 'pl

In [9]:
unpop_dict = {}

for cat in unpop_cat:
    if cat in unpop_dict:
        unpop_dict[cat] = unpop_dict[cat] + 1
    else:
        unpop_dict[cat] = 1
unpop_dict

{'': 548110,
 'there': 1485,
 'may': 4956,
 'killer': 148,
 'asteroid': 190,
 'head': 1303,
 'earth,': 130,
 'nasa': 1027,
 'decid': 977,
 'someth': 2380,
 'it.': 1888,
 'the': 39266,
 'space': 2351,
 'agenc': 847,
 'announc': 2545,
 'new': 15822,
 '"grand': 6,
 'challenge"': 2,
 'june': 1791,
 '18': 350,
 'find': 3322,
 'danger': 419,
 'rock': 561,
 'figur': 876,
 'stop': 1174,
 'destroy': 255,
 'planet.': 105,
 'mission': 713,
 'build': 2001,
 'project': 1628,
 'alreadi': 1960,
 'underway': 34,
 'nasa,': 60,
 'includ': 5244,
 'plan': 2178,
 'captur': 930,
 'asteroid,': 7,
 'pull': 786,
 'toward': 847,
 'moon': 462,
 'send': 1302,
 'astronaut': 419,
 'visit': 752,
 'As': 2413,
 'part': 3039,
 'grand': 664,
 'challenge,': 107,
 'issu': 1335,
 '"request': 5,
 'information"': 10,
 'today': 607,
 '—': 22597,
 'aim': 606,
 'solicit': 27,
 'idea': 1480,
 'industry,': 130,
 'academia': 3,
 'public': 1960,
 'improv': 858,
 'plan.': 95,
 '"we\'r': 248,
 'ask': 2465,
 'think': 3708,
 'concept':

In [10]:
pop_dict_1 = pop_dict
unpop_dict_1 = unpop_dict

In [11]:
coeff = {}

for key in pop_dict_1.keys():
    pop_num = pop_dict_1[key]
    unpop_num = 0
    if key in unpop_dict_1:
        unpop_num = unpop_dict_1.pop(key)
    coeff[key] = (pop_num-unpop_num) / (pop_num+unpop_num)
    
for key in unpop_dict_1.keys():
    coeff[key] = -1

In [12]:
X_valid.head()

,Id,Popularity,Page content
0,20000,-1,"<html><head><div class=""article-info""><span cl..."
1,20001,-1,"<html><head><div class=""article-info""><span cl..."
2,20002,1,"<html><head><div class=""article-info""><span cl..."
3,20003,-1,"<html><head><div class=""article-info""><span cl..."
4,20004,-1,"<html><head><div class=""article-info""><span cl..."


In [13]:
def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    t = article.get_text().split(' ')
    porter = PorterStemmer()
    res = [porter.stem(word) for word in t if not word in  
        stop_words]
    return res

res = []
for i in range(0, df.shape[0]):
    res.append(get_article(df['Page content'][i]))

In [14]:
cat_list = res
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[-25.934944065189626,
 23.763097211670576,
 -21.80415717349629,
 -4.107853066768798,
 3.021477455795891,
 -35.88228647907899,
 6.1891641753965,
 -6.849126554066979,
 17.65276863826323,
 -12.654792376053335,
 -15.631167090944194,
 4.710690121093586,
 3.1302253894835537,
 7.797780810009523,
 -30.093756727586367,
 -16.144980845723122,
 11.52226237236429,
 2.7102319750113173,
 15.486255552865153,
 -13.767532746209023,
 6.944387531235656,
 -17.406371476167216,
 -14.570698397133516,
 30.29999399482175,
 20.90623818719782,
 10.384115451275216,
 -15.597011647479425,
 0.045604546344459776,
 14.239256897498361,
 -15.184210115470044,
 8.145451781245004,
 3.0791089533633134,
 -7.078750129670873,
 2.6242439374958115,
 -73.4701751826296,
 -41.883486914230964,
 50.10307284179119,
 9.839121932516735,
 26.8763913432915,
 5.980154427489392,
 -28.598227329577515,
 -10.480357301069576,
 12.654593232073253,
 290.1115096647092,
 -26.42059063770815,
 38.64949174877968,
 89.78793121774339,
 -27.94749572376665

In [15]:
df["article_weight"] = weight_list

In [16]:
df.to_csv("./data/train_article_weight.csv")
df.head()

,Id,Popularity,Page content,article_weight
0,0,-1,"<html><head><div class=""article-info""> <span c...",-25.934944
1,1,1,"<html><head><div class=""article-info""><span cl...",23.763097
2,2,1,"<html><head><div class=""article-info""><span cl...",-21.804157
3,3,-1,"<html><head><div class=""article-info""><span cl...",-4.107853
4,4,-1,"<html><head><div class=""article-info""><span cl...",3.021477


In [19]:
df2 = pd.read_csv('./data/test.csv')
print(df2.head())
print(df2.shape)

      Id                                       Page content
0  27643  <html><head><div class="article-info"><span cl...
1  27644  <html><head><div class="article-info"><span cl...
2  27645  <html><head><div class="article-info"><span cl...
3  27646  <html><head><div class="article-info"><span cl...
4  27647  <html><head><div class="article-info"><span cl...
(11847, 2)


In [21]:
def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    t = article.get_text().split(' ')
    porter = PorterStemmer()
    res = [porter.stem(word) for word in t if not word in  
        stop_words]
    return res

res2 = []
for i in range(0, df2.shape[0]):
    res2.append(get_article(df2['Page content'][i]))

In [22]:
cat_list = res2
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[1.3183498527907387,
 -0.06659522517991436,
 -4.647032651274345,
 2.968519552671643,
 3.5355726007176633,
 -1.2217233364181135,
 -9.25220010637149,
 3.647572613495163,
 7.529548250533735,
 3.267807688616552,
 -2.9594718659801145,
 -2.035205758755832,
 36.64170041373665,
 -8.657029968259076,
 -4.068916846495861,
 -7.735782423562404,
 4.261798051067987,
 -2.89974812523666,
 -85.2998130308012,
 -3.4330779302376024,
 -2.3159643882641334,
 1.268059794529327,
 -2.0280237814909863,
 2.1252132505112318,
 -5.004792725766185,
 -8.118393925246338,
 5.166741017303022,
 51.338400033018665,
 -1.9344532859417758,
 -6.0297331785399,
 0.7396507216029702,
 3.072932489591127,
 4.212511214782451,
 8.173220565417887,
 -8.10493785323241,
 -15.933837950279624,
 -3.4071002897300446,
 1.8738781343241593,
 -22.01827101227776,
 19.283581852803444,
 10.261926872273383,
 -3.887286863142294,
 15.18898961413406,
 5.509685594893561,
 2.4547622431626457,
 0.0669519887100875,
 -3.958588885746994,
 -1.0469555722460044,


In [88]:
df2["article_weight"] = weight_list

In [23]:
df2.to_csv("./data/test_article_weight.csv")
df2.head()

,Id,Page content
0,27643,"<html><head><div class=""article-info""><span cl..."
1,27644,"<html><head><div class=""article-info""><span cl..."
2,27645,"<html><head><div class=""article-info""><span cl..."
3,27646,"<html><head><div class=""article-info""><span cl..."
4,27647,"<html><head><div class=""article-info""><span cl..."


In [82]:
df["cat_weight"] = weight_list

In [91]:
df.to_csv("./data/train_cat_weight.csv")

In [71]:
cat_list = res
weight_list = []
for cats in cat_list:
    weight = 0
    for cat in cats:
        if cat in coeff:
            weight = weight + coeff[cat]
    weight_list.append(weight)
    
weight_list

[-0.8387497017418277,
 -1.0902592385237577,
 0.18855292988911612,
 0.011342530607181281,
 0.23086752313855502,
 -0.81454587289996,
 -0.09618175525980585,
 -1.0319236066816422,
 -0.8941546899747144,
 -0.1769757567452964,
 -1.0261550115944087,
 -1.1428896872766254,
 0.5274096769373783,
 -0.07746401417773276,
 -0.6491216596122729,
 -0.08261022620000591,
 -1.057434038041655,
 -0.1560604072214138,
 -0.7524106597935429,
 -0.14270611717490567,
 0.3555326978742409,
 0.05380765583092719,
 -0.5460109939087803,
 0.5988517956598833,
 -0.20452965831003034,
 2.2859358114107895,
 -0.29751555947455255,
 -0.0045697871784828314,
 -0.0937335364724979,
 1.2297253567712267,
 -0.04174936326489904,
 -0.9955449547742778,
 1.0565721465309699,
 0.31961775158446143,
 0.15271746510289883,
 -1.1177224978944704,
 -0.05313924430296926,
 0.0964350617163593,
 -0.29751555947455255,
 0.012569615960219237,
 -0.009729483072938791,
 -0.5981403505268881,
 0.8665382507855257,
 0.7197226851876661,
 -0.6237064116985377,
 -1.01

In [79]:
n=0
for i in range(len(weight_list)):
    if (weight_list[i] > 0.15) and (1 == X_valid["Popularity"][i]):
        n = n+1
    elif (weight_list[i] <= 0.15) and (-1 == X_valid["Popularity"][i]):
        n = n+1
    #print(str(i) + ": \t" + str(X_valid["Popularity"][i]) +"\t"+str(weight_list[i]))
print("accu: " + str(n/len(weight_list)))

accu: 0.5304199921496795
